In [5]:
import pandas as pd
import numpy as np
import wikipedia as wp
import requests
import os
import urllib.request
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import folium
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


This is a WorkBook dedicated for MyCapStone Project

In [7]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
all_tables=soup.find_all("table")

print(soup.title.string)
right_table=soup.find('table', class_='wikitable sortable')
AA = []
BB = []
CC = []
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells) > 0:
        AA.append(cells[0].find(text=True).rstrip("\n"))
        BB.append(cells[1].find(text=True).rstrip("\n"))
        CC.append(cells[2].find(text=True).rstrip("\n"))
        
Wiki_DF_1 = pd.DataFrame(AA,columns=['Postalcode'])
Wiki_DF_1['Borough']=BB
Wiki_DF_1['Neighbourhood']=CC
print("Raw Table Size from URL : " + str(Wiki_DF_1.shape))
Wiki_DF_1.to_csv(os.getcwd() + "\\1_MyTorontoPC_RAW.csv")

A = []
B = []
C = []
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells) > 0:
        if cells[1].find(text=True).rstrip("\n") != "Not assigned":
            A.append(cells[0].find(text=True).rstrip("\n"))
            B.append(cells[1].find(text=True).rstrip("\n"))
            if cells[2].find(text=True).rstrip("\n") == "Not assigned":
                C.append(cells[1].find(text=True).rstrip("\n"))
            else:
                C.append(cells[2].find(text=True).rstrip("\n"))
        
Wiki_DF_2 = pd.DataFrame(A,columns=['Postalcode'])
Wiki_DF_2['Borough']=B
Wiki_DF_2['Neighbourhood']=C
Wiki_DF_2.to_csv(os.getcwd() + "\\2_MyTorontoPC_NA.csv")
print("Table Size after removing row's containing Borough as \"Not assigned\" : " + str(Wiki_DF_2.shape))

Wiki_DF_3 = Wiki_DF_2.groupby(['Postalcode','Borough'], as_index = False).agg({'Neighbourhood': ','.join})
Wiki_DF_3.to_csv(os.getcwd() + "\\3_MyTorontoData_GROUPED.csv")
print("Table Size after grouping row by same postalcode : " + str(Wiki_DF_3.shape))

List of postal codes of Canada: M - Wikipedia
Raw Table Size from URL : (287, 3)
Table Size after removing row's containing Borough as "Not assigned" : (210, 3)
Table Size after grouping row by same postalcode : (103, 3)


<h1>MarkDown</h1><br>
All Requested Actions Were Completed <br>
Queried Table from a Wikipedia Link into a dataframe <br>
Processed it to ensure if Borough is Not Assigned then its data is not taken into account

In [30]:
import geocoder # import geocoder
import io
url="http://cocl.us/Geospatial_data"
Geospatial_data=requests.get(url).content

A = []
B = []
C = []
D = []
E = []

Wiki_DF = Wiki_DF_3
Wiki_PostLatLon = pd.read_csv(io.StringIO(Geospatial_data.decode('utf-8')))
for index, row in Wiki_DF.iterrows():
    A.append(row['Postalcode'])
    B.append(row['Borough'])
    C.append(row['Neighbourhood'])
    
    #location = None
    #while(location is None):
    #    geolocator = Nominatim(user_agent='MyCapStone')
    #    location = geolocator.geocode({"postalcode": row['Borough']})
    LatLon = Wiki_PostLatLon[Wiki_PostLatLon["Postal Code"] == row['Postalcode']]
    D.append(LatLon["Latitude"].values[0])
    E.append(LatLon["Longitude"].values[0])

Wiki_DF_4 = pd.DataFrame(A,columns=['Postalcode'])
Wiki_DF_4['Borough']=B
Wiki_DF_4['Neighbourhood']=C
Wiki_DF_4['Latitude']=D
Wiki_DF_4['Longitude']=E

Wiki_DF_4.to_csv(os.getcwd() + "\\3_MyTorontoData_LATLON.csv")